###### Código Obsoleto

In [62]:
import pandas as pd
from slugify import slugify

from os import environ
raiz = f"{environ.get('AIRFLOW_HOME', '/home/eduardo/vscode/airflow/work')}"

### Fase Bronze

In [ ]:
mapbiomas = pd.read_excel('https://storage.googleapis.com/mapbiomas-public/brasil/coverage/col71/statistics/TABELA-GERAL-COL71-MAPBIOMAS-BIOMASxMUNICIPIOS-v2.xlsx', sheet_name = 1)


mapbiomas.to_excel(f'{raiz}/repositorio/bronze_mapbiomas_cobertura_col71.xlsx', index=False)

### Fase Prata

COLEÇÃO 7 - CLASSES					    ID
3.1. Pastagem						    15
3.2. Agricultura					    18
3.2.1. Lavoura Temporária				19
3.2.1.1. Soja						    39
3.2.1.2. Cana						    20
3.2.1.3. Arroz (beta)					40
3.2.1.4. Algodão (beta)					62
3.2.1.5. Outras Lavouras Temporárias	41
3.2.2. Lavoura Perene					36
3.2.2.1. Café						    46
3.2.2.2. Citrus						    47
3.2.1.3. Outras Lavouras Perenes		48
3.4. Mosaico de Usos					21

In [31]:
anos = (pd.read_excel(f'{raiz}/repositorio/bronze_mapbiomas_cobertura_col71.xlsx',
               usecols=lambda x: isinstance(x, int))
        .columns
        .drop(range(1985, 2000))
        .tolist()
        )

In [34]:
(pd.read_excel(f'{raiz}/repositorio/bronze_mapbiomas_cobertura_col71.xlsx',
                    converters={'city': lambda x: slugify(' '.join(x.split(' - ')[:2]))})
    .drop(range(1985, 2000), axis = 'columns')
    .assign(**{
        'city': lambda df: (df['city']).replace("santo-antonio-do-leverger-mt", "santo-antonio-de-leverger-mt")
        })
    .query('class_id in (15, 18, 19, 39, 20, 40, 62, 41, 36, 46, 47, 48, 21)') #https://mapbiomas-br-site.s3.amazonaws.com/downloads/_EN__C%C3%B3digos_da_legenda_Cole%C3%A7%C3%A3o_7.pdf
    .merge(
        pd.read_parquet(f'{raiz}/repositorio/prata_localidades.parquet')
        .assign(**{'city': lambda df: (df['municipio-nome']+' '+ df['UF-sigla']).apply(slugify)}),
        how='left',
        on='city'
        )
    .melt(
        id_vars=['UF-id', 'municipio-id', 'class_id'],
        value_vars=anos,
        var_name='ano',
        value_name='area'
        )
    .dropna(subset='area')
    .convert_dtypes()
    .rename(columns=
            {
                'UF-id':'UF',
                'municipio-id':'Municipio',
                'ano':'Ano',
                'class_id':'Classe_Solo'
             })
    .set_index(['UF', 'Municipio', 'Ano', 'Classe_Solo'])
    .to_parquet(f'{raiz}/repositorio/prata_cobert_solo_agropecuaria.parquet')
    )

### Fase Ouro

Área Destinada a Agropecuária (ha)
Elementos (kg/ha ano)

Cálculo: (Cobertura Solo Municipal * Parâmetro_X)/1000

UF e Brasil = mil t

NPK oriundo de deposição atmosférica incorporado ao solo (t)

In [2]:
parametro_N = 5.5
parametro_P = 0.3
parametro_K = 3.1

In [71]:
df = (pd.read_parquet(f'{raiz}/repositorio/prata_cobert_solo_agropecuaria.parquet',
                      use_nullable_dtypes=True)
      .assign(**{
          'N': lambda df: df['area'].apply(lambda t: t*(parametro_N/1000)),
          'P': lambda df: df['area'].apply(lambda t: t*(parametro_P/1000)),
          'K': lambda df: df['area'].apply(lambda t: t*(parametro_K/1000))
          })
      .drop('area', axis = 1)
      .convert_dtypes()
      )

In [72]:
df_UF = df.groupby(['UF', 'Ano']).sum()/1000

In [73]:
df_UF

N         P          K
UF Ano                                 
11 2000  28.824649  1.572254   16.24662
   2001  30.334594  1.654614   17.09768
   2002    32.2503  1.759107  18.177442
   2003  34.973987  1.907672  19.712611
   2004   36.84102   2.00951  20.764939
...            ...       ...        ...
53 2017   1.347048  0.073475   0.759245
   2018    1.34558  0.073395   0.758418
   2019   1.346297  0.073434   0.758822
   2020   1.348225   0.07354   0.759908
   2021   1.345631  0.073398   0.758447

[594 rows x 3 columns]

In [38]:
df_BR = df.groupby(['ano']).sum()/1000

,N,P,K
ano,,,
2000,1218.232489,66.449045,686.640130
2001,1233.090468,67.259480,695.014627
2002,1249.081624,68.131725,704.027825
2003,1272.687103,69.419297,717.332731
2004,1292.578224,70.504267,728.544090
2005,1306.828062,71.281531,736.575816
2006,1317.560608,71.866942,742.625070
2007,1323.347571,72.182595,745.886813
2008,1330.554210,72.575684,749.948736


In [61]:
with pd.ExcelWriter(f'{raiz}/repositorio/balanço_npk.xlsx', mode='a', if_sheet_exists='overlay') as ouro:
    (df_UF.assign(**{'Item': 'Deposição Atmosférica'})
     .reset_index()
     .filter(['UF', 'Ano', 'Item', 'N', 'P', 'K'])
     .convert_dtypes()
     .to_excel(
         ouro,
         'Entrada_2',
         index=False
         )
     )